# Planning heuristics : Scalar demand forecast

In [7]:
import sys
sys.path.append('D:/source/repos')
from utilities.std_imports import *
pd.set_option('display.max_columns', None)
import supply_chain.scm.inventory_planning as ip

### Plan class

In [226]:
class Plan:
    def __init__(self, horizon, dmd, on_hand):
        self.horizon = horizon
        self.plan = pd.DataFrame(columns=list(range(horizon+1)))
        self.plan.loc[0] = dmd
        self.plan.loc[1] = np.zeros(horizon+1)
        stk = np.zeros(horizon+1)
        stk[0] = on_hand
        self.plan.loc[2] = stk
        self.plan.index = ['Dmd', 'Pl.Or', 'Stock']
        
    def show(self):
        display(self.plan)       
        
    def upd_stk(self):
        for i in range(1, self.horizon+1):
            self.plan.loc['Stock'][i] = self.plan.loc['Stock'][i-1] - self.plan.loc['Dmd'][i] + self.plan.loc['Pl.Or'][i]
    
    def get_tot_dmd(self):
        return self.plan.loc['Dmd'].sum()
    
    def get_avg_stk(self):
        return round(self.plan.loc['Stock'].mean(),2)
 
    def get_hol_cost(self, hol_uc):
        stk_pos = self.plan.loc['Stock'][self.plan.loc['Stock']>0].sum()
        hol_cost = round(stk_pos * hol_uc,2)
        return hol_cost
    
    def get_tr(self, tr_fix_uc, tr_var_uc):
        po = self.plan.loc['Pl.Or'][self.plan.loc['Pl.Or']>0]
        n_po = len(po)
        tot_po = po.sum()
        tr_fix = n_po * tr_fix_uc
        tr_var = tot_po * tr_var_uc
        tr = tr_fix + tr_var
        return tr
    
    def get_tot_cost(self, hol_uc, tr_fix_uc, tr_var_uc):
        return self.get_hol_cost(hol_uc) + self.get_tr(tr_fix_uc, tr_var_uc) 

### Testing

In [227]:
dmd = np.array([0, 130, 160, 120, 260, 130, 120, 135, 105, 110, 195, 100, 110, 105, 110, 110, 115, 118, 105, 125, 122])
plan = Plan(horizon=20, dmd=dmd, on_hand=370)
plan.show()
plan.upd_stk()
plan.show()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Dmd,0,130,160,120,260,130,120,135,105,110,195,100,110,105,110,110,115,118,105,125,122
Pl.Or,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Stock,370,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Dmd,0,130,160,120,260,130,120,135,105,110,195,100,110,105,110,110,115,118,105,125,122
Pl.Or,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Stock,370,240,80,-40,-300,-430,-550,-685,-790,-900,-1095,-1195,-1305,-1410,-1520,-1630,-1745,-1863,-1968,-2093,-2215


In [228]:
tot_dmd = plan.get_tot_dmd()
avg_stk = plan.get_avg_stk()
hol_cost = plan.get_hol_cost(hol_uc=0.8)
tr_cost = plan.get_tr(tr_fix_uc=0.3, tr_var_uc=0.02)
tot_cost = plan.get_tot_cost(hol_uc=0.8, tr_fix_uc=0.3, tr_var_uc=0.02)

print('Total demand : ', tot_dmd)
print('Average stock : ', avg_stk)
print('Holding cost : ', hol_cost)
print('Transport cost : ', tr_cost)
print('Total cost : ', tot_cost)

Total demand :  2585
Average stock :  -1002.1
Holding cost :  552.0
Transport cost :  0.0
Total cost :  552.0


In [ ]:
# Minimize total cost while avoiding stockouts

In [ ]:
# display hol, tr fix , tr var and total cost per period of a Planning solution (pl orders)
# each time goes to zero if that subplan is optimal it will continue being for overall plan (?) No.
# top down: one order covering all, then split if it improves cost until it doesnt

In [ ]:
# Reorder point is pure arithmetics so quantities is the independent variable. Should be equal sum of dmds so, number of periods. Sequence of number of periods 
# until horizon

In [ ]:
# Linear programming?